In [ ]:
import pandas as pd
import numpy as np
import os
from pprint import pprint


# 1 加载数据

In [ ]:
DATA_DIR ='./submit/'
files = os.listdir(DATA_DIR)
files = [i for i in files if i[0]!='.']
print(len(files))
pprint(files)

In [ ]:
train = pd.read_csv('./data/datagrand_2021_train.csv')
#将标签进行转换
label_id2cate = dict(enumerate(train.label.unique()))
label_cate2id = {value: key for key, value in label_id2cate.items()}

# 合并所有结果
sub_exp_df = pd.read_csv('./data/datagrand_2021_test.csv')
df_merged = sub_exp_df.drop(['text'], axis=1)
for file in files:
    tmp_df = pd.read_csv(DATA_DIR + file)
    tmp_df['ylabel'] = tmp_df['label'].map(label_cate2id)
    df_merged[file] = tmp_df['label'].map(label_cate2id)
df_merged.head()

## 1.1 计算pearson相关系数

In [ ]:
x = df_merged['a.csv']
for i in range(len(files)):
    y = df_merged[files[i]]
    r = x.corr(y,method="kendall") #-0.2611165
    print(files[i],r)

# 2 投票融合

In [ ]:

# 计票
def work(pres):
    count = [0]*35
    for i in pres:
        count[i] += 1
    out = count.index(max(count))
    return out

In [ ]:
tmp_arr = np.array(df_merged.iloc[:,1:])
label_voted = [work(line) for line in tmp_arr]

In [ ]:

submit_df = sub_exp_df.drop(['text'], axis=1)
submit_df['label'] = label_voted
submit_df['label'] = submit_df['label'].map(label_id2cate)


In [ ]:
submit_df.head()

In [ ]:
model_name = "voting-a-b"
submit_df.to_csv('./voting_data/{}.csv'.format(model_name), index=False)

# 3 投票生成伪标签

In [ ]:
# 计票
def work_high(pres):
    count = [0]*35
    for i in pres:
        count[i] += 1
    p = len(files)-1 # 该数字根据融合的文件自定义的，如果有11个文件，设该值为11表示，11个结果都投票才可以作为伪标签数据
    if max(count) >p:
        out = count.index(max(count))
    else:
        out = -1
    return out

tmp_arr = np.array(df_merged.iloc[:,1:])
label_voted = [work_high(line) for line in tmp_arr]
# 没有做伪标签的数据数量
print(label_voted.count(-1))


In [ ]:
test_data =  sub_exp_df
test_data['label'] = label_voted
# 删除不能作为伪标签的数据
test_data = test_data.drop(test_data[test_data['label']==-1].index)
len(test_data)

In [ ]:
# 反编码映射
model_name = "pseudo_label_data"
test_data['label'] = test_data['label'].map(label_id2cate)
test_data

In [ ]:
test_data

In [ ]:
# 读取原始训练集
pseudo_train = train
# 合并原始训练集和伪标签数据
pseudo_label_train = pd.concat([pseudo_train,test_data]).reset_index().drop(columns=['index'])
model_name = "./data/pseudo_train_data_{}".format(len(pseudo_label_train))
pseudo_label_train.to_csv('{}.csv'.format(model_name), index=False)

In [ ]:
pseudo_label_train

In [ ]:
pseudo_label_train